In [42]:
import numpy as np 
import pandas as pd
import sqlite3

path = 'C:/Users/Harish/OneDrive/Desktop/football analysis/' 
database = path + 'database.sqlite'

In [43]:
conn = sqlite3.connect(database)
tables = pd.read_sql(""" SELECT *
                         FROM sqlite_master
                         WHERE type='table';""",conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [44]:
country = pd.read_sql("""SELECT *
                         FROM country;""",conn)
country

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [45]:
leagues = pd.read_sql("""SELECT *
                         FROM League;""",conn)
leagues

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [46]:
# Match reports

match = pd.read_sql("""SELECT *
                         FROM Match
                         LIMIT 10 ;""",conn)
match

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
5,6,1,1,2008/2009,1,2008-09-24 00:00:00,492478,8203,8342,1,...,1.67,4.35,3.40,1.70,4.50,3.40,1.70,NaN,NaN,NaN
6,7,1,1,2008/2009,1,2008-08-16 00:00:00,492479,9999,8571,2,...,3.60,2.10,3.25,3.00,1.85,3.25,3.75,2.10,3.25,3.10
7,8,1,1,2008/2009,1,2008-08-16 00:00:00,492480,4049,9996,1,...,2.40,2.80,3.25,2.25,2.80,3.20,2.25,2.88,3.25,2.20
8,9,1,1,2008/2009,1,2008-08-16 00:00:00,492481,10001,9986,1,...,3.10,2.25,3.25,2.80,2.20,3.30,2.80,2.25,3.20,2.80
9,10,1,1,2008/2009,10,2008-11-01 00:00:00,492564,8342,8571,4,...,10.00,1.30,4.35,8.50,1.25,5.00,10.00,1.29,4.50,9.00


In [47]:
# The analysis starts here
# League 

In [48]:
leages_by_season = pd.read_sql("""SELECT Country.name AS country, 
                                        League.name AS league, 
                                        season,
                                        COUNT(DISTINCT stage) AS number_of_stages,
                                        COUNT(DISTINCT HT.team_long_name) AS number_of_teams,
                                        ROUND (AVG(home_team_goal),2) AS avg_home_team_scors, 
                                        ROUND (AVG(away_team_goal),2) AS avg_away_team_goals, 
                                        ROUND (AVG(home_team_goal-away_team_goal),2) AS avg_goal_dif, 
                                        ROUND (AVG(home_team_goal+away_team_goal),2) AS avg_goals, 
                                        SUM(home_team_goal+away_team_goal) AS total_goals                                       
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                WHERE country in ('England', 'Spain', 'Germany', 'Italy', 'France')
                                GROUP BY Country.name, League.name, season
                                HAVING count(distinct stage) > 10
                                ORDER BY Country.name, League.name, season DESC
                                ;""", conn)
leages_by_season

,country,league,season,number_of_stages,number_of_teams,avg_home_team_scors,avg_away_team_goals,avg_goal_dif,avg_goals,total_goals
0,England,England Premier League,2015/2016,38,20,1.49,1.21,0.28,2.70,1026
1,England,England Premier League,2014/2015,38,20,1.47,1.09,0.38,2.57,975
2,England,England Premier League,2013/2014,38,20,1.57,1.19,0.38,2.77,1052
3,England,England Premier League,2012/2013,38,20,1.56,1.24,0.32,2.80,1063
4,England,England Premier League,2011/2012,38,20,1.59,1.22,0.37,2.81,1066
5,England,England Premier League,2010/2011,38,20,1.62,1.17,0.45,2.80,1063
6,England,England Premier League,2009/2010,38,20,1.70,1.07,0.62,2.77,1053
7,England,England Premier League,2008/2009,38,20,1.40,1.08,0.32,2.48,942
8,France,France Ligue 1,2015/2016,38,20,1.44,1.09,0.35,2.53,960
9,France,France Ligue 1,2014/2015,38,20,1.41,1.08,0.33,2.49,947


In [49]:
# Premeir League Analysis

In [50]:
premier_leages_by_season = pd.read_sql("""SELECT Country.name AS country, 
                                        League.name AS league, 
                                        season,
                                        COUNT(DISTINCT stage) AS number_of_stages,
                                        COUNT(DISTINCT HT.team_long_name) AS number_of_teams,
                                        ROUND (AVG(home_team_goal),2) AS avg_home_team_goals, 
                                        ROUND (AVG(away_team_goal),2) AS avg_away_team_goals, 
                                        ROUND (AVG(home_team_goal-away_team_goal),2) AS avg_goal_dif, 
                                        ROUND (AVG(home_team_goal+away_team_goal),2) AS avg_goals, 
                                        SUM(home_team_goal+away_team_goal) AS total_goals                                       
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                WHERE country in ('England')
                                GROUP BY season
                                ORDER BY total_goals DESC
                                ;""", conn)
premier_leages_by_season

,country,league,season,number_of_stages,number_of_teams,avg_home_team_goals,avg_away_team_goals,avg_goal_dif,avg_goals,total_goals
0,England,England Premier League,2011/2012,38,20,1.59,1.22,0.37,2.81,1066
1,England,England Premier League,2012/2013,38,20,1.56,1.24,0.32,2.80,1063
2,England,England Premier League,2010/2011,38,20,1.62,1.17,0.45,2.80,1063
3,England,England Premier League,2009/2010,38,20,1.70,1.07,0.62,2.77,1053
4,England,England Premier League,2013/2014,38,20,1.57,1.19,0.38,2.77,1052
5,England,England Premier League,2015/2016,38,20,1.49,1.21,0.28,2.70,1026
6,England,England Premier League,2014/2015,38,20,1.47,1.09,0.38,2.57,975
7,England,England Premier League,2008/2009,38,20,1.40,1.08,0.32,2.48,942


In [41]:
premier_league_teams_by_season = pd.read_sql("""SELECT  season,
                                        HT.team_long_name AS home_team, 
                                        AT.team_long_name AS away_team, 
                                        MAX (home_team_goal) AS ht_max_goals,
                                        MIN (home_team_goal) AS ht_min_goals,
                                        MAX (away_team_goal) AS at_max_goals,
                                        MIN (away_team_goal) AS at_min_goals,
                                        ROUND(AVG(home_team_goal),2) AS avg_home_team_goals, 
                                        ROUND(AVG(away_team_goal),2) AS avg_away_team_goals, 
                                        ROUND(AVG(home_team_goal-away_team_goal),2) AS avg_goal_dif, 
                                        ROUND(AVG(home_team_goal+away_team_goal),2) AS avg_goals, 
                                        SUM(home_team_goal+away_team_goal) AS total_goals                                        
                                FROM Match
                                JOIN Country on Country.id = Match.country_id
                                JOIN League on League.id = Match.league_id
                                LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
                                LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
                                WHERE Country.name = 'England'
                                GROUP BY season, home_team, away_team
                                ORDER BY total_goals  DESC
                                ;""", conn)
premier_league_teams_by_season

,season,home_team,away_team,ht_max_goals,ht_min_goals,at_max_goals,at_min_goals,avg_home_team_goals,avg_away_team_goals,avg_goal_dif,avg_goals,total_goals
0,2009/2010,Tottenham Hotspur,Wigan Athletic,9,9,1,1,9.0,1.0,8.0,10.0,10
1,2011/2012,Manchester United,Arsenal,8,8,2,2,8.0,2.0,6.0,10.0,10
2,2012/2013,Arsenal,Newcastle United,7,7,3,3,7.0,3.0,4.0,10.0,10
3,2012/2013,West Bromwich Albion,Manchester United,5,5,5,5,5.0,5.0,0.0,10.0,10
4,2009/2010,Chelsea,Sunderland,7,7,2,2,7.0,2.0,5.0,9.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...
3035,2015/2016,Watford,Southampton,0,0,0,0,0.0,0.0,0.0,0.0,0
3036,2015/2016,Watford,West Bromwich Albion,0,0,0,0,0.0,0.0,0.0,0.0,0
3037,2015/2016,West Bromwich Albion,Aston Villa,0,0,0,0,0.0,0.0,0.0,0.0,0
3038,2015/2016,West Bromwich Albion,Southampton,0,0,0,0,0.0,0.0,0.0,0.0,0
